In [ ]:
import pandas as pd
from datasets import load_dataset_builder, get_dataset_config_names, load_dataset
from huggingface_hub import HfApi
import numpy as np
import pdb
import logging
import re
from translate.storage.tmx import tmxfile
import matplotlib.pyplot as plt

# 0. Hugging Face login
- Necessary only for 'Gated' datasets on hugging face
- Specific to user (if I request access for a gated dataset, you'll need to request access also)

In [ ]:
from huggingface_hub import login
login()

## 1. Data Visualization

In [ ]:
from utils import create_conversion_dict, normalize_pairs, list_languages, list_languagesG

In [ ]:
# Read language pair data from both sources
mt_hf_df = pd.read_csv('data/language_pairs_hf.csv')
mt_ext_df = pd.read_csv('data/language_pairs_external.csv')
mt = pd.concat([mt_hf_df, mt_ext_df])
mt.head(3)

In [ ]:
print(f"Unique pairs before normalization: {len(mt['Language Pair'].unique())}")
print(f"Unique datasets: {len(mt['Author/Dataset'].unique())}")

In [ ]:
iso_mappings = create_conversion_dict()

In [ ]:
def normalize_pairs(mt_df, iso_map) -> pd.DataFrame:
    """
    Normalizes language pairs.
    1. Strips away script/locale information.
    2. Normalizes src/tgt direction
    """
    scripts = r'_[A-Z][a-z]{3}'
    endings = r'(-|_)[A-Z]{2,}'
    misc = "(-sursilv|-vallader|-tw|-valencia|_br|_tw)"

    for regex in [scripts, endings, misc]:
        mt_df['Language Pair'] = mt_df['Language Pair'].str.replace(regex, "", regex=True)
    
    mt_df['Language Pair'] = mt_df['Language Pair'].str.replace(r'2', "-", regex=True)
    
    stragglers = mt_df['Language Pair'].str.split('-', expand=True)
    stragglers = stragglers.iloc[:, 2:]
    stragglers = stragglers[stragglers.notna().any(axis=1)]
    mt_df = mt_df.drop(stragglers.index)

    mt_df[['lang_1', 'lang_2']] = mt_df['Language Pair'].str.split('-', expand=True)
    mt_df['lang_1'] = mt_df['lang_1'].apply(lambda x: iso_mappings.get(x, x))
    mt_df['lang_2'] = mt_df['lang_2'].apply(lambda x: iso_mappings.get(x, x))    
    mt_df['Language Pair'] = mt_df.apply(lambda row: f"{tuple((row['lang_1'], row['lang_2']))}", axis=1)
    
    missing_langs = mt_df[mt_df['lang_1'].isna() | mt_df['lang_2'].isna()]
    mt_df = mt_df.drop(missing_langs.index)
    
    mt_df['Language Pair'] = mt_df.apply(lambda row: f"{min(row['lang_1'], row['lang_2'])}-{max(row['lang_1'], row['lang_2'])}", axis=1)
    
    return mt_df

In [ ]:
norm_mt = normalize_pairs(mt, iso_mappings)

In [ ]:
print(f"Unique pairs: {len(norm_mt['Language Pair'].unique())}")
print(f"Unique datasets: {len(norm_mt['Author/Dataset'].unique())}") # one dataset dropped

In [ ]:
norm_mt.head()

In [ ]:
supp_Glang = list_languages()
supp_Glang_v2 = list_languagesG()

In [ ]:
supp_Glang.update(supp_Glang_v2)

In [ ]:
def is_in_Google(row, supported): 
    return row['lang_1'] in supported and row['lang_2'] in supported

In [ ]:
test = norm_mt[~norm_mt.apply(lambda x: is_in_Google(row=x,supported=supp_Glang), axis=1)]
almost = test.sort_values(by='# Train Set', ascending=False).copy()

In [ ]:
almost.head(30)

In [ ]:
mt.head()
mt['total n_examples'] = mt['# Train Set'] + mt['# Development Set'] + mt['# Test Set']
mt.head(1)

In [ ]:
group_a = mt[mt['total n_examples'] > nice_to_have]
group_c = mt[mt['total n_examples'] < needs_work]
group_b = mt[(mt['total n_examples'] <= nice_to_have) & (mt['total n_examples'] >= needs_work)]

In [ ]:
assert len(group_b) + len(group_a) + len(group_c) == len(mt) # nice!

In [ ]:
test = data[(data['# Development Set'] == 0) & (data['# Test Set'] == 0)]

In [ ]:
test = test[test['# Train Set'] < 1000000]

In [ ]:
almost = test.sort_values(by='# Train Set', ascending=False).copy()

In [ ]:
def is_in_google_translate(row):
    lang_1, lang_2 = row["Language Pair"].split("-")
    return lang_1 in iso_codes and lang_2 in iso_codes

# Apply the function to filter out rows
df_cleaned = almost[~almost.apply(is_in_google_translate, axis=1)]

In [ ]:
df_cleaned.head(10)

In [ ]:
# df_cleaned=df_cleaned.drop(3249)

In [ ]:
x_data = df_cleaned.head(10)['Language Pair']
x_data

In [ ]:
x_data = [
    "Asturian-Spanish",
    "Norwegian Bokmål-Russian",
    "French-Plateau Malagasy",
    "Bodo-English",
    "Russian-Veps",
    "French-Kabyle",
    "English-Bodo",
    "English-Kashmiri",
    "Plateau Malagasy-Russian",
    "Montenegrin-English"
]


In [ ]:
x_data = [
    "Asturian-XX",
    "Norwegian Bokmål-XX",
    "XX-Plateau Malagasy",
    "Bodo-XX",
    "XX-Veps",
    "XX-Kabyle",
    "XX-Bodo",
    "XX-Kashmiri",
    "Plateau Malagasy-XX",
    "Montenegrin-XX"
]

In [ ]:
y_data = df_cleaned.head(10)['# Train Set']

In [ ]:
y_data

In [ ]:
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#d62728', '#e377c2', '#2ca02c', '#7f7f7f']

In [ ]:
plt.figure(figsize=(14, 8))
bars = plt.bar(x_data, y_data, width=0.8, color=colors)  # Adjust width of bars

# Add labels and title
plt.xlabel('Language Pairs')
plt.ylabel('Number of Examples')
plt.title('Unsupported languages in Google Translate')
plt.ticklabel_format(style='plain', axis='y')  # Ensure y-axis is not in scientific notation

# Add value labels above bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height, f'{height:,}', 
             ha='center', va='bottom', fontsize=7, fontweight='bold')

# Adjust x-tick labels
plt.xticks(rotation=23, ha='right', fontsize=10, fontstyle='italic')  # Rotate and adjust font size

# Show the plot
plt.tight_layout()  # Ensure the layout is adjusted to prevent overlap
plt.show()


In [ ]:
plt.figure(figsize=(14, 8))
bars = plt.bar(x_data, y_data, color='blue')

# Add labels and title
plt.xlabel('Language Pairs')
plt.ylabel('Number of Examples')
plt.title('Language Pairs: Where to next?')
plt.ticklabel_format(style='plain', axis='y')  # Ensure y-axis is not in scientific notation

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height, f'{height:,}', 
             ha='center', va='bottom', fontsize=8, fontweight='bold')

# Show the plot
plt.xticks(rotation=15, fontstyle='italic')
plt.show()

In [ ]:
x_data

In [ ]:
y_data

In [ ]:
languages_a = set()

for row in group_a.iterrows():
    languages_a.add(row[1][5])
    languages_a.add(row[1][6])
    
languages_b = set()

for row in group_b.iterrows():
    languages_b.add(row[1][5])
    languages_b.add(row[1][6])
    
languages_c = set()

for row in group_c.iterrows():
    languages_c.add(row[1][5])
    languages_c.add(row[1][6])

In [ ]:
print(f"group a: {len(languages_a)}\ngroup b: {len(languages_b)}\ngroup c: {len(languages_c)}")

In [ ]:
mt_set = languages_a | languages_b | languages_c
print(f"Unique languages: {len(mt_set)}") # unique LANGUAGES

In [ ]:
a_b = languages_a | languages_b

In [ ]:
len(a_b)

In [ ]:
a_b_c = languages_a | languages_b | languages_c

In [ ]:
len(a_b_c)

In [ ]:
7100 - len(a_b_c)

In [ ]:
print(f"n_datasets for group a: {len(group_a['Author/Dataset'].unique())}")
print(f"n_datasets for group b: {len(group_b['Author/Dataset'].unique())}")
print(f"n_datasets for group c: {len(group_c['Author/Dataset'].unique())}")
print(f"n_datasets overall: {len(mt['Author/Dataset'].unique())}")

In [ ]:
test = pd.read_csv('data/mt_hf.csv')

In [ ]:
test2=test[test['Dataset Type'].str.contains('parallel', case=False, na=False)]

In [ ]:
# possible mt datasets
len(test2) # APPROX. DATASETS

## 2. Update ```language_pairs_external.csv```

In [ ]:
from utils import update_pairs
from math import factorial

In [ ]:
mtex_pair = pd.read_csv('data/language_pairs_external.csv')
mtex_pair.head()

In [ ]:
# Take a look at the helper function for ```update pairs```
help(update_pairs)

### Multiway example using  [HornMT](https://github.com/asmelashteka/HornMT) dataset from GitHub
The number of language pairs for a multiway is obtained with the permutation formula.
- Change save to True to save your changes

In [ ]:
def permutations(n, r):
    '''Returns the number of permutations.'''
    return int(factorial(n) / factorial(n-r))

val = permutations(6, 2)
print(f"There will be {val} distinct pairs for the HornMT dataset.")

In [ ]:
# Arguments for function
data_auth = 'LesanAI/HornMT' # if external check the main contributor to the dataset
langs = ['aa', 'am', 'en', 'om', 'so', 'ti']
rows = [0, 0, 2030] # multiway datasets will have the same n_rows
d_type = 'Multiway'
save = False # change to True; param is False only for demonstration purposes

In [ ]:
df = update_pairs(data_auth, langs, rows, d_type, save)
df.tail(30) # 30 distinct pairs

### English-Centric example using  [Samantar](https://huggingface.co/datasets/ai4bharat/samanantar) dataset from Hugging Face
There will be *n-1* number of language pairs for an English-Centric dataset. There will be 11 unique pairs for Samantar.
- If the dataset doesn't exist in the ```mt_hf.csv``` dataset then you will manually add the dataset to ```mt_hf_external.csv```

In [ ]:
# Arguments for function
data_auth = 'ai4bharat/samanantar' # if external check the main contributor to the dataset
configs = get_dataset_config_names(data_auth)
print(configs)

In [ ]:
langs = configs.copy()
langs.append('en') # ensure English is in the list
d_type = 'English-Centric'
save = False # change to True; param is False only for demonstration purposes
print(langs)

English-Centric datasets may not have the same n_rows! Therefore we'll create a dictionary for each unique language pair containing their (train, validation, test) pair.

In [ ]:
# this is easy if the config is similar to Samantar 
pairs = {}
for config in configs:
    rows = [0, 0, 0]
    builder = load_dataset_builder(data_auth, config)
    info = builder.info
    for split in info.splits:
        if split.startswith('train'):
            rows[0] = info.splits[split].num_examples
        if split.startswith('val'):
            rows[1] = info.splits[split].num_examples
        if split.startswith('test'):
            rows[2] = info.splits[split].num_examples
            
    pairs[config] = rows

In [ ]:
# Otherwise you'll have to manually enter the number of rows or think of a programmatic solution.
test = {}
test['as'] = [141226, 0, 0]
test['bn'] = [8604579, 0, 0]

In [ ]:
print("There will be 11 distinct pairs for the Samanantar dataset.")

In [ ]:
df = update_pairs(data_auth, langs, pairs, d_type, save)
df.tail(11) 

### Simple parallel example using  [Filtered-Japanese-English-Parallel-Corpus](https://github.com/asmelashteka/HornMT) dataset from Hugging Face
A simple parallel dataset contains only 2 language pairs.

In [ ]:
data_auth = 'Moleys/Filtered-Japanese-English-Parallel-Corpus' # if external check the main contributor to the dataset
langs = ['ja', 'en']
rows = [10739509, 0, 0] 
d_type = 'Simple Parallel'
save = False # change to True; param is False only for demonstration purposes

In [ ]:
df = update_pairs(data_auth, langs, rows, d_type, save)
df.tail(1)

In [ ]:
dataset_name = 'FBK-MT/mGeNTE'

In [ ]:
configs = get_dataset_config_names(dataset_name)
configs

In [ ]:
ds = load_dataset(dataset_name, configs[1])
# builder = load_dataset_builder()

In [ ]:
ds

In [ ]:
# data_auth = 'FBK-MT/gender-bias-PE' # if external check the main contributor to the dataset
data_auth = dataset_name
langs = ['en', 'it']
rows = [0, 0, 1500] 
d_type = 'Simple Parallel'
save = True # change to True; param is False only for demonstration purposes

In [ ]:
df = update_pairs(data_auth, langs, rows, d_type, save)
df.tail(1)

In [ ]:
df.tail()

## 3. Misc

In [ ]:
with open('Belgium_justice.tmx', 'r') as fin:
    file = tmxfile(fin, 'nl', 'fr')

In [ ]:
count = 0
for node in file.unit_iter():
    count += 1
#     print(node.source, node.target)

In [ ]:
count

In [ ]:
# re.fullmatch(pattern, configs[12])

In [ ]:
import re

# Define the pattern for basic ISO language code pairs
# pattern = r'^[a-z]{2,3}(-|2)[a-z]{2,3}$'
pattern = r'[a-z]{2,3}((_|-)\w+)?(-|2)[a-z]{2,3}((_|-)\w+)?' # new pattern!

# Example language code pairs
codes = ['en-es', 'fr-de', 'zh-en', 'EN-es', 'eng-es_AM', 'ara_blahblah', 'iwslt14_de_en', 'amh_Ethi-arb_Arab']

# Filter valid codes
valid_codes = [code for code in codes if re.fullmatch(pattern, code)]

print(valid_codes)  # Output: ['en-es', 'fr-de', 'zh-en']


In [ ]:
pattern = r'[a-z]{2,3}-[a-z]{2,3}(_-)?.*'
# pattern = 'en-zh'

In [ ]:
string = "aya_dataset"
re.search(pattern, string)
# help(re.match)